In [ ]:
import xarray as xr
import hvplot.xarray
import cartopy.crs as ccrs
import pandas as pd

from timutils import describe_environment

In [ ]:
describe_environment.print_env()

In [ ]:
ds = xr.open_dataset('/global/cscratch1/sd/twhilton/WRFv4.1_Experiments/BTRAN_tests/BTRAN_test_001/WRFV4/run/output/BTRAN.nc')

In [ ]:
ds['XLAT'].attrs['long_name'] = 'latitude'
ds['XLONG'].attrs['long_name'] = 'longitude'
ds['XLAT'].attrs['units'] = 'deg N'
ds['XLONG'].attrs['units'] = 'deg E'

In [ ]:
# convert timestamps from numpy.datetime64 to pandas.Timestamp.  
# hvplot seems to crash and burn on datetime64.
ds = ds.rename_vars({'XTIME': 'Time'})
ds = ds.assign_coords({'Time': pd.to_datetime(ds.Time.values)})
# remove time dimension from spatial coordinates.  It's constant anyway, 
# and holoviz doesn't like it.
ds = ds.assign({'latitude': ds.XLAT.sel(Time=ds.Time[0]),
                'longitude': ds.XLONG.sel(Time=ds.Time[0])})
ds = ds.set_coords(('latitude', 'longitude'))

In [ ]:
map_options = {'x': 'longitude',
               'y': 'latitude',
               'coastline': '50m',
               'projection': ccrs.PlateCarree(),
               'groupby': 'Time',
               'fields': {'Time': {'default': ds['Time'][-1].values}}}

soil water transpiration factor varies from 0.0 to 1.0; 0.0 is wilting.

In [ ]:
theplot = ds['BTRAN'].hvplot.quadmesh(**map_options,
                                      clim=(0, 1),
                                      title=ds['BTRAN'].attrs['description'])
theplot

In [ ]:
delta_btran = ds['BTRAN'].diff(dim='Time')
delta_btran.name = '\u0394BTRAN'
tstr = '\u0394 from previous tstep; {}'.format(ds['BTRAN'].attrs['description'])
theplot = delta_btran.hvplot.quadmesh(title=tstr,
                                      clim=(-1, 1),
                                      cmap="RdBu",
                                      **map_options)
theplot